## ETL 

Iniciamos la extracción, transformación y carga de nuestros datos. Nos dirigimos hacía un enfoque, donde la data que procesemos, nos ayuda en el desarrollo de nuestros kpi´s, así como en la exploración estadistica.

In [3]:
#Importamos las librerías que usaremos
import pandas as pd  
import numpy as np
import datetime

Funciones utilizadas 

In [4]:
def encontrar_datos_nulos(dataframe):
    # Verificar la presencia de valores nulos en el DataFrame
    datos_nulos = dataframe.isnull()

    # Contar la cantidad de valores nulos en cada columna
    resumen_nulos = datos_nulos.sum()

    # Filtrar las columnas que tienen al menos un valor nulo
    columnas_con_nulos = resumen_nulos[resumen_nulos > 0]

    # Imprimir el resumen
    print("Resumen de datos nulos:")
    if columnas_con_nulos.empty:
        print("No hay datos nulos en el DataFrame.")
    else:
        print(columnas_con_nulos)


def eliminar_nulos(dataframe):
    # Eliminar todos los registros que contienen al menos un valor NaN
    dataframe_sin_nan = dataframe.dropna(axis=0)

    return dataframe_sin_nan

def eliminar_columnas(dataframe, columnas_a_eliminar):
    # Eliminar las columnas especificadas
    dataframe_sin_columnas = dataframe.drop(columnas_a_eliminar, axis=1, inplace=False)

    return dataframe_sin_columnas

def convertir_columna_float_a_int(dataframe, nombre_columna):
    # Verificar si la columna existe en el DataFrame
    if nombre_columna not in dataframe.columns:
        print(f"La columna '{nombre_columna}' no existe en el DataFrame.")
        return dataframe

    # Convertir la columna de float a int y reemplazar los valores nulos con 0
    dataframe[nombre_columna] = dataframe[nombre_columna].fillna(0).astype(int)

def porcentaje_datos_nulos_por_columna(dataframe):
    """
    Calcula el porcentaje de datos nulos por columna en un DataFrame.

    Parámetros:
    - dataframe: DataFrame de pandas

    Retorna:
    - Un DataFrame que muestra el porcentaje de datos nulos por columna
    """
    # Calcular el porcentaje de datos nulos por columna
    porcentaje_nulos = (dataframe.isnull().sum() / len(dataframe)) * 100

    # Crear un nuevo DataFrame con los resultados
    resultado = pd.DataFrame({'Columna': dataframe.columns, 'Porcentaje_Nulos': porcentaje_nulos})

    return resultado







Cargamos nuestros archivos

In [5]:
#Nota: en excle se cargaron los archivos para una exploración preliminar, encontrando datos nulos como SD, sd, SD-SD entre otros
# Definir nombres de archivos y hojas de cálculo
archivo_homicidios = '../DatosCrudos/homicidios.xlsx'
archivo_lesiones = '../DatosCrudos/lesiones.xlsx'
hoja_hechos = 'HECHOS'
hoja_victimas = 'VICTIMAS'
valores_nulos = ['SD', 'sd']

# Cargar los archivos
homicidios = pd.read_excel(archivo_homicidios, sheet_name=hoja_hechos, na_values=valores_nulos)
vic_hom = pd.read_excel(archivo_homicidios, sheet_name=hoja_victimas, na_values=valores_nulos)
lesiones = pd.read_excel(archivo_lesiones, sheet_name=hoja_hechos, na_values=valores_nulos)
vic_les = pd.read_excel(archivo_lesiones, sheet_name=hoja_victimas, na_values=valores_nulos)



Comenzaremos limpiando los archivos df_homicidios y df_vic_hom, ya que pertenecen al mismo archivo

In [6]:
vic_hom

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19.0,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70.0,2016-01-02 00:00:00
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30.0,2016-01-03 00:00:00
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18.0,NaN
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29.0,2016-02-01 00:00:00
...,...,...,...,...,...,...,...,...,...,...
712,2021-0092,2021-12-12,2021,12,12,PEATON,PEATON,FEMENINO,50.0,2021-12-12 00:00:00
713,2021-0093,2021-12-13,2021,12,13,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,18.0,2021-12-18 00:00:00
714,2021-0094,2021-12-20,2021,12,20,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,43.0,2021-12-20 00:00:00
715,2021-0095,2021-12-30,2021,12,30,CONDUCTOR,MOTO,MASCULINO,27.0,2022-01-02 00:00:00


In [7]:
homicidios

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4.0,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1.0,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7.0,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0.0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,NaN
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5.0,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691,2021-0093,1,2021-12-13,2021,12,13,17:10:00,17.0,AV. RIESTRA Y MOM,AVENIDA,...,NaN,MOM,RIESTRA AV. y MOM,7,Point (102728.60090138 98186.24929177),-58.43353773,-34.64561636,MOTO-AUTO,MOTO,AUTO
692,2021-0094,1,2021-12-20,2021,12,20,01:10:00,1.0,AU DELLEPIANE Y LACARRA,AUTOPISTA,...,NaN,LACARRA AV.,"DELLEPIANE, LUIS, TTE. GRAL. y LACARRA AV.",9,Point (99624.29795829 97569.69801131),-58.46739825,-34.65117757,MOTO-AUTO,MOTO,AUTO
693,2021-0095,1,2021-12-30,2021,12,30,00:43:00,0.0,AV. GAONA Y TERRADA,AVENIDA,...,NaN,TERRADA,GAONA AV. y TERRADA,11,Point (99116.45492358 101045.23284826),-58.47293407,-34.61984745,MOTO-CARGAS,MOTO,CARGAS
694,2021-0096,1,2021-12-15,2021,12,15,10:30:00,10.0,AV. EVA PERON 4071,AVENIDA,...,4071.0,NaN,"PERON, EVA AV. 4071",9,Point (99324.54463985 97676.26932409),-58.47066794,-34.65021673,AUTO-CARGAS,AUTO,CARGAS


Analizamos los tipos de datos

In [8]:
homicidios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   695 non-null    object        
 7   HH                     695 non-null    float64       
 8   LUGAR_DEL_HECHO        695 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  Cruce                  525 non-null    object        
 13  Direc

In [9]:
encontrar_datos_nulos(homicidios) #Probamos nuestra función

Resumen de datos nulos:
HORA                       1
HH                         1
LUGAR_DEL_HECHO            1
Calle                      1
Altura                   567
Cruce                    171
Dirección Normalizada      8
VICTIMA                    9
ACUSADO                   23
dtype: int64


## Con este analisis en mente, comenzaremos la limpieza del primer archivo

Se puede asumir que SD en este dataframe es sin dato, por ende, nulo. Por lo que procedemos a cambiar los SD por nulos. De igual manera, en la columna COMUNA procedemos con los 0.

In [10]:
# Reemplazar "SD" por np.nan en todo el DataFrame
homicidios.replace("SD", np.nan, inplace=True)
homicidios.replace("SD-SD", np.nan, inplace=True)

# remplazar la comuna 0 a nulos
homicidios.loc[homicidios['COMUNA'] == '0' ,'COMUNA'] = np.nan


# remplazar los "." por nulos en latitud y longitud
homicidios.loc[homicidios['pos x'] == '.' ,'pos x'] = np.nan
homicidios.loc[homicidios['pos y'] == '.' ,'pos y'] = np.nan
homicidios.loc[homicidios['XY (CABA)'] == 'Point (. .)' ,'XY (CABA)'] = np.nan

In [11]:
homicidios

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4.0,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8.0,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1.0,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9.0,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7.0,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1.0,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0.0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8.0,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,NaN
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5.0,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1.0,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691,2021-0093,1,2021-12-13,2021,12,13,17:10:00,17.0,AV. RIESTRA Y MOM,AVENIDA,...,NaN,MOM,RIESTRA AV. y MOM,7.0,Point (102728.60090138 98186.24929177),-58.43353773,-34.64561636,MOTO-AUTO,MOTO,AUTO
692,2021-0094,1,2021-12-20,2021,12,20,01:10:00,1.0,AU DELLEPIANE Y LACARRA,AUTOPISTA,...,NaN,LACARRA AV.,"DELLEPIANE, LUIS, TTE. GRAL. y LACARRA AV.",9.0,Point (99624.29795829 97569.69801131),-58.46739825,-34.65117757,MOTO-AUTO,MOTO,AUTO
693,2021-0095,1,2021-12-30,2021,12,30,00:43:00,0.0,AV. GAONA Y TERRADA,AVENIDA,...,NaN,TERRADA,GAONA AV. y TERRADA,11.0,Point (99116.45492358 101045.23284826),-58.47293407,-34.61984745,MOTO-CARGAS,MOTO,CARGAS
694,2021-0096,1,2021-12-15,2021,12,15,10:30:00,10.0,AV. EVA PERON 4071,AVENIDA,...,4071.0,NaN,"PERON, EVA AV. 4071",9.0,Point (99324.54463985 97676.26932409),-58.47066794,-34.65021673,AUTO-CARGAS,AUTO,CARGAS


Tratamos los datos categoricos

In [12]:
#Convertimos las columnas a tipo categorico
categorical_columns = ['HH', 'TIPO_DE_CALLE', 'COMUNA', 'ACUSADO']
homicidios[categorical_columns] = homicidios[categorical_columns].astype('category')

#convertimos 'pos x' y 'pos y' en numerico
numeric_columns = ['pos x', 'pos y']
homicidios[numeric_columns] = homicidios[numeric_columns].astype('float')


In [13]:
#Buscamos nulos con una de nuestras funciones 
encontrar_datos_nulos(homicidios)

Resumen de datos nulos:
HORA                       1
HH                         1
LUGAR_DEL_HECHO            1
Calle                      1
Altura                   567
Cruce                    171
Dirección Normalizada      8
XY (CABA)                 14
pos x                     12
pos y                     12
PARTICIPANTES              5
VICTIMA                    9
ACUSADO                   23
dtype: int64


In [14]:
homicidios

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4.0,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8.0,Point (98896.78238426 93532.43437792),-58.475340,-34.687570,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1.0,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9.0,Point (95832.05571093 95505.41641999),-58.508775,-34.669777,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7.0,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1.0,Point (106684.29090040 99706.57687843),-58.390403,-34.631894,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0.0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8.0,Point (99840.65224780 94269.16534422),-58.465039,-34.680930,MOTO-SD,MOTO,NaN
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5.0,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1.0,Point (106980.32827929 100752.16915795),-58.387183,-34.622466,MOTO-PASAJEROS,MOTO,PASAJEROS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691,2021-0093,1,2021-12-13,2021,12,13,17:10:00,17.0,AV. RIESTRA Y MOM,AVENIDA,...,NaN,MOM,RIESTRA AV. y MOM,7.0,Point (102728.60090138 98186.24929177),-58.433538,-34.645616,MOTO-AUTO,MOTO,AUTO
692,2021-0094,1,2021-12-20,2021,12,20,01:10:00,1.0,AU DELLEPIANE Y LACARRA,AUTOPISTA,...,NaN,LACARRA AV.,"DELLEPIANE, LUIS, TTE. GRAL. y LACARRA AV.",9.0,Point (99624.29795829 97569.69801131),-58.467398,-34.651178,MOTO-AUTO,MOTO,AUTO
693,2021-0095,1,2021-12-30,2021,12,30,00:43:00,0.0,AV. GAONA Y TERRADA,AVENIDA,...,NaN,TERRADA,GAONA AV. y TERRADA,11.0,Point (99116.45492358 101045.23284826),-58.472934,-34.619847,MOTO-CARGAS,MOTO,CARGAS
694,2021-0096,1,2021-12-15,2021,12,15,10:30:00,10.0,AV. EVA PERON 4071,AVENIDA,...,4071.0,NaN,"PERON, EVA AV. 4071",9.0,Point (99324.54463985 97676.26932409),-58.470668,-34.650217,AUTO-CARGAS,AUTO,CARGAS


Después hacemos una purga. Para nuestro proposito, las columnas 'FECHA', 'DD', 'HORA', 'LUGAR_DEL_HECHO', 'Calle', 'Altura', 'Cruce', 'Dirección Normalizada', no nos sirven

In [15]:
columnas_a_eliminar=['FECHA', 'DD', 'HORA', 'LUGAR_DEL_HECHO', 'Calle', 'Altura', 'Cruce']


Para reducir la cantidad de accidentes mortales para motocicletas en un 7% son datos necesarios 'PARTICIPANTES', 'VICTIMAS', 'ACUSADOS' así como la columna 'MM' para nuestro analisis estadistico.
A su vez para reducir en un 10% la tasa de homicidios en siniestros viales de los últimos seis meses, en CABA, en comparación con la tasa de homicidios en siniestros viales del semestre anterior, necesitamos las columnas antes mencionadas. Haciendo un hincapie en la columna 'MM'.

In [16]:
#Aplicamos nuestra función 
homicidiosLimpio=eliminar_columnas(homicidios, columnas_a_eliminar)

De esta manera, concluimos el tratamiento de nulo. Es momento de analizar los tipos de datos

In [17]:
homicidiosLimpio.tail(2)

,ID,N_VICTIMAS,AAAA,MM,HH,TIPO_DE_CALLE,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
694,2021-0096,1,2021,12,10.0,AVENIDA,"PERON, EVA AV. 4071",9.0,Point (99324.54463985 97676.26932409),-58.470668,-34.650217,AUTO-CARGAS,AUTO,CARGAS
695,2021-0097,1,2021,11,6.0,CALLE,PADRE CARLOS MUGICA 709,1.0,Point (107664.16647795 104708.63962087),-58.379762,-34.586796,BICICLETA-AUTO,BICICLETA,AUTO


In [18]:
#Iniciamos con otro informe, teniendo en cuenta nuestro objetivo final
homicidiosLimpio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   ID                     696 non-null    object  
 1   N_VICTIMAS             696 non-null    int64   
 2   AAAA                   696 non-null    int64   
 3   MM                     696 non-null    int64   
 4   HH                     695 non-null    category
 5   TIPO_DE_CALLE          696 non-null    category
 6   Dirección Normalizada  688 non-null    object  
 7   COMUNA                 696 non-null    category
 8   XY (CABA)              682 non-null    object  
 9   pos x                  684 non-null    float64 
 10  pos y                  684 non-null    float64 
 11  PARTICIPANTES          691 non-null    object  
 12  VICTIMA                687 non-null    object  
 13  ACUSADO                673 non-null    category
dtypes: category(4), float64(2), int64(3), obje

Ahora procedemos con el segundo

In [19]:
vic_hom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_hecho             717 non-null    object        
 1   FECHA                717 non-null    datetime64[ns]
 2   AAAA                 717 non-null    int64         
 3   MM                   717 non-null    int64         
 4   DD                   717 non-null    int64         
 5   ROL                  706 non-null    object        
 6   VICTIMA              708 non-null    object        
 7   SEXO                 711 non-null    object        
 8   EDAD                 664 non-null    float64       
 9   FECHA_FALLECIMIENTO  649 non-null    object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(5)
memory usage: 56.1+ KB


In [20]:
# Definir las columnas a eliminar
columnas_a_eliminar = ['AAAA', 'MM', 'DD', 'FECHA_FALLECIMIENTO']

# Eliminar columnas no deseadas
vic_hom.drop(columns=columnas_a_eliminar, inplace=True)

In [21]:
# Crear una columna con la condición 'GRAVEDAD' igual a 'FATAL'
vic_hom['GRAVEDAD'] = 'FATAL'

## ETL - Victimas

Luego de una limpieza en nuestro primer archivo, es hora de limpiar nuestro segundo archivo, siempre teniendo en mente el producto final

In [22]:
#Exploramos nuestro archivo
lesiones.tail(15)

,id,n_victimas,aaaa,mm,dd,fecha,hora,franja_hora,direccion_normalizada,comuna,...,latutid,victima,acusado,participantes,moto,auto,transporte_publico,camion,ciclista,gravedad
23770,LC-2021-0652171,1,2021,12,31,2021-12-31 00:00:00,05:15:00,5.0,"CORDOBA AV. y SCALABRINI ORTIZ, RAUL AV.",15,...,-3.459430e+01,NaN,NaN,SD-SD,NaN,NaN,NaN,NaN,NaN,NaN
23771,LC-2021-0652203,2,2021,12,31,2021-12-31 00:00:00,07:00:00,7.0,YATAY y DIAZ VELEZ AV.,5,...,-3.460873e+01,NaN,NaN,SD-SD,NaN,NaN,NaN,NaN,NaN,NaN
23772,LC-2021-0652278,1,2021,12,31,2021-12-31 00:00:00,08:25:00,8.0,NaN,NaN,...,NaN,NaN,NaN,SD-SD,NaN,NaN,NaN,NaN,NaN,NaN
23773,LC-2021-0652316,5,2021,12,31,2021-12-31 00:00:00,09:00:00,9.0,"3 DE FEBRERO y PEDRAZA, MANUELA",13,...,-3.454839e+01,AUTO,NaN,AUTO-SD,NaN,x,NaN,NaN,NaN,NaN
23774,LC-2021-0652398,1,2021,12,31,2021-12-31 00:00:00,08:45:00,8.0,MEXICO y SANTIAGO DEL ESTERO,1,...,-3.461544e+01,NaN,NaN,SD-SD,NaN,NaN,NaN,NaN,NaN,NaN
23775,LC-2021-0652473,1,2021,12,31,2021-12-31 00:00:00,10:30:00,10.0,"FIGUEROA ALCORTA, PRES. AV. y AUSTRIA",2,...,-3.458168e+01,NaN,NaN,SD-SD,NaN,NaN,NaN,NaN,NaN,NaN
23776,LC-2021-0652565,1,2021,12,31,2021-12-31 00:00:00,11:10:00,11.0,RIVADAVIA AV. 11500,9,...,-3.463932e+01,NaN,NaN,SD-SD,NaN,NaN,NaN,NaN,NaN,NaN
23777,LC-2021-0652677,1,2021,12,31,2021-12-31 00:00:00,13:10:00,13.0,"MORENO, JOSE MARIA AV. 63",6,...,-3.461903e+01,NaN,NaN,SD-SD,NaN,NaN,NaN,NaN,NaN,NaN
23778,LC-2021-0652681,1,2021,12,31,2021-12-31 00:00:00,14:00:00,14.0,"BELAUSTEGUI, LUIS, DR. y GONZALEZ, JOAQUIN V.",11,...,-3.462188e+01,NaN,NaN,SD-SD,NaN,NaN,NaN,NaN,NaN,NaN
23779,LC-2021-0652812,1,2021,12,31,2021-12-31 00:00:00,18:05:00,18.0,VIEL 276,6,...,-3.461970e+01,NaN,NaN,SD-SD,NaN,NaN,NaN,NaN,NaN,NaN


Podemos notar, como en el archivo anterior, SD, SD-SD, x entre otros representan los datos nulos. Procedemos, con esto en mente, a cambiar estos datos a NaN para proseguir con la limpieza y tratamientos de nulos 

In [23]:
# Lista de valores a considerar como nulos
valores_nulos = ['SD', 'SD-SD', 'x', 0, 0.0]

# Reemplazar los valores indicados con NaN
lesiones.replace(valores_nulos, np.nan, inplace=True)

In [25]:
lesiones

,id,n_victimas,aaaa,mm,dd,fecha,hora,franja_hora,direccion_normalizada,comuna,...,latutid,victima,acusado,participantes,moto,auto,transporte_publico,camion,ciclista,gravedad
0,LC-2019-0000179,1.0,2019,1,1,2019-01-01 00:00:00,09:00:00,9.0,NaN,14,...,-3.455966e+01,CICLISTA,NaN,CICLISTA-SD,NaN,NaN,NaN,NaN,NaN,NaN
1,LC-2019-0000053,1.0,2019,1,1,2019-01-01 00:00:00,01:55:00,1.0,NaN,8,...,-3.466913e+01,AUTO,NaN,AUTO-SD,NaN,NaN,NaN,NaN,NaN,NaN
2,LC-2019-0000063,1.0,2019,1,1,2019-01-01 00:00:00,02:00:00,2.0,NaN,8,...,-3.467756e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,LC-2019-0000079,1.0,2019,1,1,2019-01-01 00:00:00,02:30:00,2.0,NaN,7,...,-3.464735e+01,PEATON,NaN,PEATON-SD,NaN,NaN,NaN,NaN,NaN,NaN
4,LC-2019-0000082,4.0,2019,1,1,2019-01-01 00:00:00,04:30:00,4.0,NaN,3,...,-3.460458e+01,AUTO,NaN,AUTO-SD,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23780,LC-2021-0652849,1.0,2021,12,31,2021-12-31 00:00:00,19:30:00,19.0,"ALBERDI, JUAN BAUTISTA AV. y GUAMINI",9,...,-3.465971e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23781,LC-2021-0652865,2.0,2021,12,31,2021-12-31 00:00:00,19:40:00,19.0,"ALBERDI, JUAN BAUTISTA AV. 4436",9,...,-3.464175e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23782,LC-2021-0652907,1.0,2021,12,31,2021-12-31 00:00:00,20:00:00,20.0,NaN,1,...,-3.458308e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23783,LC-2021-0652921,1.0,2021,12,31,2021-12-31 00:00:00,22:00:00,22.0,LINIERS VIRREY y MORENO,5,...,-3.461429e+01,MOTO,TRANSPORTE PUBLICO,MOTO-TRANSPORTE PUBLICO,NaN,NaN,NaN,NaN,NaN,GRAVE


Después de este pequeño analisis, procedemos a eliminar las columnas innecesarias

In [26]:
#Eliminar las columnas que no son importantes
columnas_a_eliminar = ['aaaa', 'mm', 'dd', 'hora', 'direccion_normalizada',
                        'calle', 'altura', 'cruce', 'participantes', 'moto',
                        'auto', 'transporte_publico', 'camion', 'otra_direccion',
                        'ciclista', 'victima']

lesiones.drop(columns=columnas_a_eliminar, inplace=True)

Hacemos ahora un conteo de los datos nulos 

In [27]:
encontrar_datos_nulos(lesiones)

Resumen de datos nulos:
n_victimas                  1
franja_hora               816
comuna                   1015
tipo_calle              11045
geocodificacion_CABA     1252
longitud                 1471
latutid                  1471
acusado                 15288
gravedad                23139
dtype: int64


Aplicamos mi función para analizar el porcentaje 

In [28]:
porcentaje_datos_nulos_por_columna(lesiones)

,Columna,Porcentaje_Nulos
id,id,0.000000
n_victimas,n_victimas,0.004204
fecha,fecha,0.000000
franja_hora,franja_hora,3.430734
comuna,comuna,4.267395
tipo_calle,tipo_calle,46.436830
geocodificacion_CABA,geocodificacion_CABA,5.263822
longitud,longitud,6.184570
latutid,latutid,6.184570
acusado,acusado,64.275804


In [29]:
# Pasar las columnas a mayusculas
lesiones.columns = lesiones.columns.str.upper()

In [30]:
# Cambiar nombre de varias columnas en un solo llamado a rename
nuevos_nombres = {
    'ID': 'ID_HECHO',
    'FRANJA_HORA': 'HORA',
    'TIPO_CALLE': 'TIPO_DE_CALLE',
    'GEOCODIFICACION_CABA': 'XY (CABA)',
    'ACUSADO': 'VEHICULO_ACUSADO'
}

lesiones.rename(columns=nuevos_nombres, inplace=True)

In [31]:
lesiones

,ID_HECHO,N_VICTIMAS,FECHA,HORA,COMUNA,TIPO_DE_CALLE,XY (CABA),LONGITUD,LATUTID,VEHICULO_ACUSADO,GRAVEDAD
0,LC-2019-0000179,1.0,2019-01-01 00:00:00,9.0,14,NaN,Point (104991.46333918849995825 107720.9664748...,-5.840891e+01,-3.455966e+01,NaN,NaN
1,LC-2019-0000053,1.0,2019-01-01 00:00:00,1.0,8,NaN,Point (101813.84712503915943671 95578.55507230...,-5.844351e+01,-3.466913e+01,NaN,NaN
2,LC-2019-0000063,1.0,2019-01-01 00:00:00,2.0,8,NaN,Point (99538.58660300650808495 94643.418717990...,-5.846834e+01,-3.467756e+01,NaN,NaN
3,LC-2019-0000079,1.0,2019-01-01 00:00:00,2.0,7,NaN,Point (102372.16830963784013875 97994.13750300...,-5.843742e+01,-3.464735e+01,NaN,NaN
4,LC-2019-0000082,4.0,2019-01-01 00:00:00,4.0,3,NaN,Point (105968.98286849579017144 102737.1734686...,-5.839822e+01,-3.460458e+01,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
23780,LC-2021-0652849,1.0,2021-12-31 00:00:00,19.0,9,AVENIDA,Point (95400.5441905024 96621.6007711877),-5.851348e+01,-3.465971e+01,NaN,NaN
23781,LC-2021-0652865,2.0,2021-12-31 00:00:00,19.0,9,AVENIDA,Point (97705.4491469278 98614.8825470092),-5.848833e+01,-3.464175e+01,NaN,NaN
23782,LC-2021-0652907,1.0,2021-12-31 00:00:00,20.0,1,NaN,Point (107377.10694920316745993 105120.7873965...,-5.838289e+01,-3.458308e+01,NaN,NaN
23783,LC-2021-0652921,1.0,2021-12-31 00:00:00,22.0,5,CALLE,Point (104472.702045632 101660.930364258),-5.841453e+01,-3.461429e+01,TRANSPORTE PUBLICO,GRAVE


In [32]:
# Hacer cambios en la columna 'FECHA'
lesiones['FECHA'] = pd.to_datetime(lesiones['FECHA'].str.replace("'", ""), format='%y/%m/%d', errors='coerce').dt.date

# Reemplazar "No especificada" por nulos en la columna 'COMUNA'
lesiones['COMUNA'].replace('No Especificada', np.nan, inplace=True)

# Añadir gravedad 'LEVEMENTE HERIDO' a las filas con 'GRAVEDAD' nulo
lesiones['GRAVEDAD'].fillna('LEVEMENTE HERIDO', inplace=True)

In [33]:
lesiones

,ID_HECHO,N_VICTIMAS,FECHA,HORA,COMUNA,TIPO_DE_CALLE,XY (CABA),LONGITUD,LATUTID,VEHICULO_ACUSADO,GRAVEDAD
0,LC-2019-0000179,1.0,NaT,9.0,14.0,NaN,Point (104991.46333918849995825 107720.9664748...,-5.840891e+01,-3.455966e+01,NaN,LEVEMENTE HERIDO
1,LC-2019-0000053,1.0,NaT,1.0,8.0,NaN,Point (101813.84712503915943671 95578.55507230...,-5.844351e+01,-3.466913e+01,NaN,LEVEMENTE HERIDO
2,LC-2019-0000063,1.0,NaT,2.0,8.0,NaN,Point (99538.58660300650808495 94643.418717990...,-5.846834e+01,-3.467756e+01,NaN,LEVEMENTE HERIDO
3,LC-2019-0000079,1.0,NaT,2.0,7.0,NaN,Point (102372.16830963784013875 97994.13750300...,-5.843742e+01,-3.464735e+01,NaN,LEVEMENTE HERIDO
4,LC-2019-0000082,4.0,NaT,4.0,3.0,NaN,Point (105968.98286849579017144 102737.1734686...,-5.839822e+01,-3.460458e+01,NaN,LEVEMENTE HERIDO
...,...,...,...,...,...,...,...,...,...,...,...
23780,LC-2021-0652849,1.0,NaT,19.0,9.0,AVENIDA,Point (95400.5441905024 96621.6007711877),-5.851348e+01,-3.465971e+01,NaN,LEVEMENTE HERIDO
23781,LC-2021-0652865,2.0,NaT,19.0,9.0,AVENIDA,Point (97705.4491469278 98614.8825470092),-5.848833e+01,-3.464175e+01,NaN,LEVEMENTE HERIDO
23782,LC-2021-0652907,1.0,NaT,20.0,1.0,NaN,Point (107377.10694920316745993 105120.7873965...,-5.838289e+01,-3.458308e+01,NaN,LEVEMENTE HERIDO
23783,LC-2021-0652921,1.0,NaT,22.0,5.0,CALLE,Point (104472.702045632 101660.930364258),-5.841453e+01,-3.461429e+01,TRANSPORTE PUBLICO,GRAVE


Ahora vamos con el sub archivo

In [34]:
# Cambiar el nombre de las columnas y estandarizarlas
nuevos_nombres = {'ID_hecho': 'ID_HECHO', 'ROL': 'ROL_VICTIMA', 'VICTIMA': 'VEHICULO_VICTIMA',
                  'SEXO': 'SEXO_VICTIMA', "EDAD": "EDAD_VICTIMA"}

# Cambiar el nombre de la columna 'EDAD'
vic_les.rename(columns={"EDAD": "EDAD_VICTIMA"}, inplace=True)

# Remplazar nombres de columnas
vic_les.rename(columns=nuevos_nombres, inplace=True)

# Mostrar las columnas del DataFrame después del cambio
vic_les.columns


Index(['ID hecho', 'AAA', 'MM', 'DD', 'FECHA ', 'VEHICULO_VICTIMA',
       'SEXO_VICTIMA', 'EDAD_VICTIMA', 'GRAVEDAD'],
      dtype='object')

In [35]:
vic_les

,ID hecho,AAA,MM,DD,FECHA,VEHICULO_VICTIMA,SEXO_VICTIMA,EDAD_VICTIMA,GRAVEDAD
0,LC-2019-0000053,2019,1,1,2019-01-01,NaN,Varon,57.0,NaN
1,LC-2019-0000063,2019,1,1,2019-01-01,NaN,NaN,NaN,NaN
2,LC-2019-0000079,2019,1,1,2019-01-01,NaN,Varon,NaN,NaN
3,LC-2019-0000082,2019,1,1,2019-01-01,NaN,Varon,45.0,NaN
4,LC-2019-0000082,2019,1,1,2019-01-01,NaN,Mujer,45.0,NaN
...,...,...,...,...,...,...,...,...,...
27600,LC-2021-0451911,2021,9,11,2021-09-11,TRANSPORTE PUBLICO,Varon,87.0,NaN
27601,LC-2021-0530228,2021,10,25,2021-10-25,TRANSPORTE PUBLICO,Mujer,60.0,NaN
27602,LC-2021-0530228,2021,10,25,2021-10-25,TRANSPORTE PUBLICO,Mujer,32.0,NaN
27603,LC-2021-0201378,2021,5,2,2021-05-02,MOTO,Varon,32.0,NaN


In [ ]:
# Cambia el formato de la columna 'FECHA'
vic_les['FECHA'] = pd.to_datetime(vic_les['FECHA'], errors='coerce').dt.date


In [37]:
vic_les.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27605 entries, 0 to 27604
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ID hecho          27605 non-null  object        
 1   AAA               27605 non-null  int64         
 2   MM                27605 non-null  int64         
 3   DD                27605 non-null  int64         
 4   FECHA             27605 non-null  datetime64[ns]
 5   VEHICULO_VICTIMA  11333 non-null  object        
 6   SEXO_VICTIMA      25377 non-null  object        
 7   EDAD_VICTIMA      23134 non-null  float64       
 8   GRAVEDAD          537 non-null    object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(4)
memory usage: 1.9+ MB


Exportamos nuestro archivos

In [71]:
lesiones.to_csv("../Limpios/Lesiones.csv")
vic_les.to_csv("../Limpios/VictimasLesiones.csv")

## Motociclistas

Armamos un nuevo dataframe únicamente con las victimas "moto" para nuestro kpi

In [26]:
vic_hom

,ID_hecho,FECHA,ROL,VICTIMA,SEXO,EDAD,GRAVEDAD
0,2016-0001,2016-01-01,CONDUCTOR,MOTO,MASCULINO,19.0,FATAL
1,2016-0002,2016-01-02,CONDUCTOR,AUTO,MASCULINO,70.0,FATAL
2,2016-0003,2016-01-03,CONDUCTOR,MOTO,MASCULINO,30.0,FATAL
3,2016-0004,2016-01-10,CONDUCTOR,MOTO,MASCULINO,18.0,FATAL
4,2016-0005,2016-01-21,CONDUCTOR,MOTO,MASCULINO,29.0,FATAL
...,...,...,...,...,...,...,...
712,2021-0092,2021-12-12,PEATON,PEATON,FEMENINO,50.0,FATAL
713,2021-0093,2021-12-13,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,18.0,FATAL
714,2021-0094,2021-12-20,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,43.0,FATAL
715,2021-0095,2021-12-30,CONDUCTOR,MOTO,MASCULINO,27.0,FATAL


In [28]:
df_moto = vic_hom[vic_hom['VICTIMA'].str.contains(r'moto', case=False, na=False, regex=True)]


In [29]:
df_moto

,ID_hecho,FECHA,ROL,VICTIMA,SEXO,EDAD,GRAVEDAD
0,2016-0001,2016-01-01,CONDUCTOR,MOTO,MASCULINO,19.0,FATAL
2,2016-0003,2016-01-03,CONDUCTOR,MOTO,MASCULINO,30.0,FATAL
3,2016-0004,2016-01-10,CONDUCTOR,MOTO,MASCULINO,18.0,FATAL
4,2016-0005,2016-01-21,CONDUCTOR,MOTO,MASCULINO,29.0,FATAL
5,2016-0008,2016-01-24,CONDUCTOR,MOTO,MASCULINO,30.0,FATAL
...,...,...,...,...,...,...,...
709,2021-0089,2021-12-02,CONDUCTOR,MOTO,MASCULINO,41.0,FATAL
711,2021-0091,2021-12-11,CONDUCTOR,MOTO,MASCULINO,24.0,FATAL
713,2021-0093,2021-12-13,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,18.0,FATAL
714,2021-0094,2021-12-20,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,43.0,FATAL
